# 1) Exploration des donnés

**Hypothèses**:
- Dans le readme expliquant les data, pour le fichier "Imbalances_Prices_Case.csv", je pense qu'il y a une erreur pour l'explication de la variable "pos_imb_settlement_price", qui équivaut au cas où les ventes sont inférieures à l'injection et non l'inverse; seule la dernière phrase serait donc incorrecte et tout le reste serait correct. Celà me semble cohérent que le prix dinjection d'un surplus sur le réseau soit plus faible que le prix de rachat d'un surplus de vente. 
- Les ventes sont faites au lendemain (day-ahead) et donc chaque jour à 23h30 heure locale, les ordres de vente doivent être passés pour toute la journée du lendemain, de 0h00 à 23h30. Dans ce contexte, nous prendrons un *leadtime* de 24h (chaque jour) pour les calculs de la nMAE et du coût des écarts, et ce *leadtime* correspondra au pas de temps 0h00 heure locale pour chaque journée du dataset (temps auquel la nMAE et le coût des écarts peuvent être calculés en fonction des données de production/prévision/prix des dernières 24h)
- Le choix est fait de considérer uniquement les heures locales au lieu des heures UTC. Ce choix est fait car le dataset démarre le 01/10/2021 à 0h00 heure locale, ce qui correspond à un lead time; les 1ères valeurs de nMAE seront donc calculées le 02/10/2021 à 0h00, en prenant en compte les 24h précédentes. Le problème de ce choix est que lors du changement d'heures le 31/10/2021, 2 pas de temps (31/10/2021 2h00 et 31/10/2021 2h30) sont dédoublés. Le choix est donc fait de ne pas prendre en compte cette dernière journée et de prendre le dernier leadtime le 31/10/2021 à 0h00 heure locale.

importation des librairies python

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objects as go
from datetime import timedelta, datetime
from tqdm import tqdm

pd.set_option('display.max_rows', None) # pas de limite d'affichage en nb de lignes pour les dataframes

chargement des 4 fichiers de données dans des dataframes

In [ ]:
da_prices = pd.read_csv("./data/DA_Prices_Case.csv")
imbalance_prices = pd.read_csv("./data/Imbalances_Prices_Case.csv")
power_case = pd.read_csv("./data/Power_Case.csv")
power_forecasts = pd.read_csv("./data/Power_Forecasts_Case.csv")

passage des colonnes concernées en type "datetime"

In [ ]:
# for da_prices
date_columns = ["start_ts_utc","end_ts_utc","start_ts", "end_ts"]
for col in date_columns:
    da_prices[col] = pd.to_datetime(da_prices[col])
# for imbalances_prices
date_columns = ["start_ts_utc","end_ts_utc","start_ts", "end_ts"]
for col in date_columns:
    imbalance_prices[col] = pd.to_datetime(imbalance_prices[col])
# for power_case
date_columns = ["start_ts_utc","end_ts_utc","start_ts", "end_ts"]
for col in date_columns:
    power_case[col] = pd.to_datetime(power_case[col])
# for power_forecasts
date_columns = ["start_ts_utc","end_ts_utc","start_ts", "end_ts", "comp_ts_utc"]
for col in date_columns:
    power_forecasts[col] = pd.to_datetime(power_forecasts[col])

## 2) 1ère analyse des fichiers de donnée

***imbalances prices***

In [ ]:
imbalance_prices.head(5)

In [ ]:
first_date_utc = imbalance_prices["start_ts_utc"][0]
last_date_utc = imbalance_prices["start_ts_utc"].iloc[-1]
first_date_local = imbalance_prices["start_ts"][0]
last_date_local = imbalance_prices["start_ts"].iloc[-1]
print(f"first time step (utc) -> {first_date_utc}")
print(f"last time step (utc) -> {last_date_utc}")
print(f"first time step (local time) -> {first_date_local}")
print(f"last time step (local time) -> {last_date_local}")

affichage des données sur toute la période (graphe intéractif, posibilité de zoomer)

In [ ]:
fig = px.line(imbalance_prices, x='start_ts_utc', y=["pos_imb_settlement_price","neg_imb_settlement_price"])
fig.update_yaxes(title_text='Price (€ / MWh)')
fig.show()

affichage de la répartition des prix sur la période donnée via un boxplot

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=imbalance_prices["pos_imb_settlement_price"],name="pos prices"))
fig.add_trace(go.Box(y=imbalance_prices["neg_imb_settlement_price"],name="neg prices"))
fig.update_yaxes(title_text='Price (€ / MWh)')

Analyse:
- Les prix de rachat et vente par le gestionnaire du réseau sont fournis au pas de temps de 30 minutes, du 30/09/2021 22h->22h30 au 31/10/2021 22h->22h30
- les prix d'achat semblent être toujours supérieurs aux prix de vente, ce qui est logique pour inciter les producteurs à ne pas survendre (vendre plus que leur production)

***day ahead prices***

In [ ]:
da_prices.head(5)

In [ ]:
first_date = da_prices["start_ts_utc"][0]
last_date = da_prices["start_ts_utc"].iloc[-1]
print(f"first time step (utc) -> {first_date}")
print(f"last time step (utc) -> {last_date}")

affichage des données sur toute la période (graphe intéractif, posibilité de zoomer)

In [ ]:
fig = px.line(da_prices, x='start_ts_utc', y='price')
fig.update_yaxes(title_text='Price (€ / MWh)')
fig.show()

affichage de la répartition des prix sur la période donnée via un boxplot

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=da_prices["price"],name="DA prices"))
fig.update_yaxes(title_text='Price (€ / MWh)')

Analyse:
- Les prix day ahead sont fournis au pas de temps de 30 minutes, du 30/09/2021 22h->22h30 au 31/10/2021 22h->22h30

***power case***

In [ ]:
power_case.head(5)

In [ ]:
first_date = da_prices["start_ts_utc"][0]
last_date = da_prices["start_ts_utc"].iloc[-1]
print(f"first time step (utc) -> {first_date}")
print(f"last time step (utc) -> {last_date}")

affichage des données par centrale, sur toute la période (graphe intéractif, posibilité de zoomer)

In [ ]:
fig = px.line(power_case,x="start_ts_utc", y="power", color="utility")
fig.update_yaxes(title_text='Price (€ / MWh)')
fig.show()

affichage de la répartition des puissances mesurées sur la période donnée, par actif, via un boxplot

In [ ]:
fig = px.box(power_case, x="utility", y="power")
fig.update_yaxes(title_text='Power (kW)')
fig.show()

Analyse:
- Les puissances mesurées sont fournies au pas de temps de 30 minutes, du 30/09/2021 22h->22h30 au 31/10/2021 22h->22h30
- sur les 5 premières lignes du tableau, on constate qu'il n'y a pas de données de production pour la centrale n°3 le 30/09/2021 entre 22h et 22h30. Soit la centrale n'était pas encore active à ce moment là, soit il y a eu un problème lors de la récupértion des mesures.

***power forecasts***

In [ ]:
power_forecasts.head(5)

In [ ]:
first_date = power_forecasts["comp_ts_utc"][0]
last_date = power_forecasts["comp_ts_utc"].iloc[-1]
print(f"first time step (utc) -> {first_date}")
print(f"last time step (utc) -> {last_date}")

affichage des données par centrale, sur toute la période (graphe intéractif, posibilité de zoomer)

In [ ]:
# fig = px.line(power_forecasts,x="start_ts_utc", y="power", color="utility")
# fig.update_yaxes(title_text='Price (€ / MWh)')
# fig.show()

ce graphe n'est pas lisible en l'état, car pour chaque pas de temps et chaque centrale, les prévisions sont données plusieurs fois, sur les 4 jours précédents... un pre-processing est nécessaire.

affichage de la répartition des puissances prédites sur la période donnée, par actif, via un boxplot

In [ ]:
fig = px.box(power_forecasts, x="utility", y="power")
fig.update_yaxes(title_text='Power (kW)')
fig.show()

Analyse:
- Les puissances prédites sont fournies au pas de temps de 30 minutes, du 30/09/2021 22h->22h30 au 31/10/2021 22h->22h30
- sur les 5 premières lignes du tableau on constate que l'on a des valeurs de puissance prédites pour la centrale n°3 (mais pas de puissance mesurée comme vu précédemment). La centrale devait donc être en fonctionnement à ce moment là, mais il y a probablement eu des erreurs pour récupérer les mesures.

# 3) Aggrégation des données par actif

**aggrégation des données de production**

Création d'un dataframe avec en index "start_ts" et une colonne par centrale, indiquant la puissance produite.

L'idée est d'abord de créer une colonne avec tous les pas de temps de 30min entre le 01/10/2021 0h00 et le 31/10/2021 00h00, puis de merger les productions de chaque actif un par un en mettant des valeurs "Nan" si pas de valeur pour un pas de temps.


In [ ]:
# création d'un dataframe avec la puissance produite pour chaque actif
prod_a0 = power_case[power_case["utility"]==0][["start_ts","power"]]
prod_a1 = power_case[power_case["utility"]==1][["start_ts","power"]]
prod_a2 = power_case[power_case["utility"]==2][["start_ts","power"]]
prod_a3 = power_case[power_case["utility"]==3][["start_ts","power"]]
prod_a4 = power_case[power_case["utility"]==4][["start_ts","power"]]
prods=[prod_a0,prod_a1,prod_a2,prod_a3,prod_a4]

# création d'un dataframe avec tous les pas de temps de 30min sur la période concernée
prod_data = pd.DataFrame(pd.date_range(start="2021-10-01 00:00:00",end="2021-10-31 0:00:00",freq="30min"))
prod_data.columns=["start_ts"]

prod_data.shape

In [ ]:
# rajout des colonnes pour chaque actif
for i in range(5):
    prod_data = prod_data.merge(prods[i],how="left",on="start_ts")
    prod_data.columns = ["start_ts"] + [f"prod_{j}" for j in range(i+1)]
    
# affichage des 5 premières lignes u dataframe obtenu
prod_data.head()

**aggrégation des données de prévision de production**

modifications préalables:
- la colonne "comp_ts_utc" est convertie en heure locale
- les données sont conservées uniquement jusqu'au 31/10/2021 à 0h00 heure locale (colonne "start_ts")

In [ ]:
%%capture
agg_forecasts = power_forecasts[(power_forecasts['start_ts'] >= "2021-10-01 00:00:00") & (power_forecasts['start_ts'] <= "2021-10-31 0:00:00")]
agg_forecasts["comp_ts_utc"]=agg_forecasts["comp_ts_utc"].apply(lambda x: x + timedelta(hours=2))
agg_forecasts.columns = ['Unnamed: 0', 'start_ts_utc', 'end_ts_utc', 'start_ts', 'end_ts',
       'comp_ts_local', 'utility', 'power']

création d'un dataframe de prévisions par centrale

In [ ]:
# création d'un dataframe avec la puissance produite pour chaque actif
pred_a0 = agg_forecasts[agg_forecasts["utility"]==0][["start_ts","comp_ts_local","power"]]
pred_a1 = agg_forecasts[agg_forecasts["utility"]==1][["start_ts","comp_ts_local","power"]]
pred_a2 = agg_forecasts[agg_forecasts["utility"]==2][["start_ts","comp_ts_local","power"]]
pred_a3 = agg_forecasts[agg_forecasts["utility"]==3][["start_ts","comp_ts_local","power"]]
pred_a4 = agg_forecasts[agg_forecasts["utility"]==4][["start_ts","comp_ts_local","power"]]
preds = [pred_a0, pred_a1, pred_a2, pred_a3, pred_a4]

Pour chaque actif et pour chaque pas de temps, nous avons des prévisions faites chaque heure lors des 3 jours précédents.

Une hypothèse forte ici est de retenir, pour chaque pas de temps, uniquement la valeur de prédiction obtenue lors du dernier leadtime.

Par exemple pour le pas de temps 01/10/2021 4h30, nous retenons la valeur prédite le 30/09/2021 à 23h30 (la prévision se fait toutes les heures et non toutes les 30min, donc il n'y a pas de valeur pour 0h00).

Une autre idée, avec plus de temps, aurait été de prendre la moyenne des prévisions entre la plus vieille prédiction et la prédiction du leadtime précédent.

In [ ]:
# creation d'une liste des pas de temps pour lesquels on veut une prévision de production
time_values = list(pd.date_range(start="2021-10-01 00:00:00",end="2021-10-31 0:00:00",freq="30min"))

# parcours des dataframes de chaque centrale et création d'un dataframe par centrale avec uniquement la valeur
# de production prévue pour chaque pas de temps (celle de la veille à 23h30)
id = 0
dataframes = []
for pred_df in preds:
    timestamps=[]
    powers = []
    idx = 0
    for val in tqdm(time_values):
        timestamps.append(val)
        df = pred_df[pred_df['start_ts'] == val]
        time = val - timedelta(days=1)
        time = time.replace(hour=23, minute=30)
        power = df[df["comp_ts_local"]==time]
        if not power.empty:
            power = power["power"].iloc[0].item()
        else:
            power = np.nan # si pas de valeur de pred, on met "nan"
        powers.append(power)
        idx += 1
    dataframes.append(pd.DataFrame({"start_ts": timestamps, "power_pred": powers}))
    id += 1

# aggrégation des 5 dataframes obtenus en un dataframe finale, avec une colonne par actif
pred_data = pd.DataFrame(pd.date_range(start="2021-10-01 00:00:00",end="2021-10-31 0:00:00",freq="30min"))
pred_data.columns=["start_ts"]
for i in range(5):
    pred_data = pred_data.merge(dataframes[i],how="left",on="start_ts")
    pred_data.columns = ["start_ts"] + [f"pred_{j}" for j in range(i+1)]
    
# affichage des 5 premières lignes du dataframe obtenu
pred_data.head()

# 4) Jointure des différentes sources de données

Création d'un dataframe final, avec les colonnes suivantes:
- **start_ts:** début du pas de temps, en heure locale
- **prod_i:** la production de l'actif i, en **kW**
- **pred_i:** la prévision de production pour l'actif i, en **kW**
- **prix_day_ahead:** le prix day ahead, en **€/MWh**
- **pos_imb_price:** le prix de rachat par le gestionnaire de réseau pour le surplus de production de Boralex, en **€/MWh**
- **neg_imb_price:** le prix de vente par le gestionnaire de réseau pour le manque en production de Boralex, en **€/MWh**

**jointure des différentes données**

In [ ]:
final_data = prod_data.merge(pred_data,how="left",on="start_ts")
final_data = final_data.merge(da_prices[["start_ts","price"]],how="left",on="start_ts")
final_data = final_data.merge(imbalance_prices[["start_ts","pos_imb_settlement_price","neg_imb_settlement_price"]],how="left",on="start_ts")
final_data.columns=['start_ts', 'prod_0', 'prod_1', 'prod_2', 'prod_3', 'prod_4', 'pred_0','pred_1', 'pred_2', 'pred_3', 'pred_4', 'prix_day_ahead','pos_imb_price', 'neg_imb_price']

# affichage des 5 premières lignes du dataframe
print(f"shape -> {final_data.shape}")
final_data.head()

**analyse des valeurs manquantes**

*prévisions de production*

In [ ]:
fig = px.line(final_data, x='start_ts' ,y=["pred_0","pred_1","pred_2","pred_3","pred_4"])
fig.update_yaxes(title_text='Price (€ / MWh)')
fig.show()

on constate qu'il manque les valeurs de prévisions, pour les 5 actifs, du 23/10/2021 au 25/10/2021 inclus.

*production*

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(rows=5, cols=1,subplot_titles=("prod_0","prod_1","prod_2","prod_3","prod_4"))
fig.add_trace(go.Scatter(x=final_data["start_ts"], y=final_data["prod_0"]),row=1,col=1)
fig.add_trace(go.Scatter(x=final_data["start_ts"], y=final_data["prod_1"]),row=2,col=1)
fig.add_trace(go.Scatter(x=final_data["start_ts"], y=final_data["prod_2"]),row=3,col=1)
fig.add_trace(go.Scatter(x=final_data["start_ts"], y=final_data["prod_3"]),row=4,col=1)
fig.add_trace(go.Scatter(x=final_data["start_ts"], y=final_data["prod_4"]),row=5,col=1)
fig.update_layout(height=900, width=1400, showlegend=False)
fig.show()

en faisant un zoom au niveau des zones de trou des graphiques ci-dessus, on constate que l'on peut supprimer les journées suivantes (car trop de données manquantes):
- **prod_0:** le 03/10/2021, le 09/10/2021, le 12/10/2021, le 22/10/2021
- **prod_1:** le 12/10/2021 (déjà supprimé)
- **prod_2:** le 03/10/2021 (déjà supprimé), le 04/10/2021
- **prod_3** le 01/10/2021, le 30/10/2021
- **prod_4:** le 01/10/2021 (déjà supprimé), le 28/10/2021

**Résumé des valeurs à supprimer**

avec plus de temps, un travail plus fin sur les valeurs manquantes pourrait être réalisé, mais dans le contexte de ce test, les journées suivantes sont supprimées:

le 01/10/2021, 03/10/2021, le 09/10/2021, le 12/10/2021, du 22/10/2021 au 25/10/2021 inclus, le 28/10/2021, le 30/10/2021

In [ ]:
# suppression des journées avec données manquantes
def bad_day(row):
    return row["start_ts"].day in [1,3,9,12,22,23,24,25,28,30]  and row["start_ts"].month == 10
final_data["to_delete"] = final_data.apply(lambda row: bad_day(row), axis=1)
final_data = final_data[final_data["to_delete"]==False]
final_data = final_data.drop(labels="to_delete", axis=1)

après suppression, on passe de 1441 à 961 lignes, donc avec plus de temps un travail plus précis pour traiter les données manquantes est nécessaire 

In [ ]:
final_data.info()

Des valeurs sont encore manquantes localement, elles sont remplacées par la valeur moyenne de chaque variable.

Certaines valeurs sont négatives, elles sont donc remplacées par des valeurs nulles.

Ici encore, un travail plus précis serait réalisable avec plus de temps.

In [ ]:
%%capture
final_data = final_data.fillna(final_data.mean())
num_columns = final_data._get_numeric_data()
num_columns[num_columns<0]=0

In [ ]:
final_data.info()

nous avons notre dataframe final pour le calcul des métriques.

# 5) Calcul des 2 métriques

Les données de production et de prévision sont fournies au pas de temps 30min sur une période d'environ 1 mois, du 30/09/2021 au 31/10/2021, pour 5 centrales différentes.
Comme vu précédemment, certaines journées sont retirées de l'étude du fait d'un nombre de données trop faible.

La nMAE et le coût des écarts seront calculés à chaque leadtime, selon les formules suivantes:

Pour la **nMAE**:
$$
nMAE=\frac{1}{48P_{inst}}\sum_{i=t_0-48}^{t_0}|P_{prod}-P_{pred}|
$$

Pour le **coût des écarts**:
$$
C_{ecarts}=\sum_{i=t_0-48}^{t_0}(V_{ref}-V_{nette})
$$
Avec, pour chaque pas de temps de 30min:
$$
V_{ref}=P_{prod}C_{DayAhead}
$$
Si la puissance produite est supérieure à la puissance vendue:
$$
V_{nette}=(P_{vendue}C_{DayAhead}+P_{diff}C_{PosImbPrice})
$$
Si la puissance produite est inférieure à la puissance vendue:
$$
V_{nette}=(P_{vendue}C_{DayAhead}-P_{diff}C_{NegImbPrice})
$$


Comme indiqué dans l'énoncé, **la puissance vendue est toujours égale à la puissance prédite**.

**Calcul de la nMAE**

In [ ]:
def bad_day(row):
    return row["start_ts"].day in [1,3,9,12,22,23,24,25,28,30]  and row["start_ts"].month == 10
final_data["to_delete"] = final_data.apply(lambda row: bad_day(row), axis=1)

In [ ]:
total_capacity = 93.7

In [ ]:
time_values = [x for x in final_data.start_ts if (x.hour==0 and x.minute==0)]
n_maes = []
prods = []
for time in tqdm(time_values):
    start = time
    stop = time.replace(hour=23,minute=30)
    df = final_data[(final_data["start_ts"]>=str(start)) & (final_data["start_ts"]<=str(stop))]
    mae = 0
    for index, row in df.iterrows():
        prod = row[1:6].sum()/1000 # conversion des kW en MW
        prods.append(prod)
        pred = row[7:12].sum()/1000 # conversion des kW en MW
        mae += np.abs(prod-pred)
    n_mae = round(mae/(total_capacity * 48) * 100,2) # car il y a 48 pas de temps de 30min par jour
    n_maes.append(n_mae)

**Calcul du coût des écarts**

In [ ]:
couts_ecarts = []
for time in tqdm(time_values):
    start = time
    stop = time.replace(hour=23,minute=30)
    df = final_data[(final_data["start_ts"]>=str(start)) & (final_data["start_ts"]<=str(stop))]
    cout_ecarts = 0
    for index, row in df.iterrows():
        prod = row[1:6].sum()/2000 # conversion des kW en MWh (/1000 pour kW -> MW and /2 pour le pas de temps de 1/2 heure)
        pred = row[7:12].sum()/2000 # conversion des kW en MWh (/1000 pour kW -> MW and /2 pour le pas de temps de 1/2 heure)
        v_ref = prod * row.prix_day_ahead
        if prod > pred:
            v_nette = pred * row.prix_day_ahead + (prod - pred) * row.pos_imb_price
        else:
            v_nette = pred * row.prix_day_ahead - (pred - prod) * row.neg_imb_price
        cout_ecarts += v_ref - v_nette
    cout_ecart = round(cout_ecarts,2) # car il y a 48 pas de temps de 30min par jour
    couts_ecarts.append(cout_ecart)

**synthèse**

Les 2 métriques, calculés au pas de temps journalier, sont rassemblées dans un dataframe: dont les colonnes sont les suivantes:
- **start_ts:** leadtime pour le calcul des métriques
- **n_mae:** nMAE pour les 24 heures précédent le leadtime, en **%**
- **cout_ecarts:** coût des écarts pour les 24h précédent le leadtime, en **€**

In [ ]:
metrics_df = pd.DataFrame({"start_ts":time_values, "n_mae":n_maes ,"cout_ecarts":couts_ecarts})
metrics_df

Les résultats sont analysés ci-après via 2 graphiques.
Ne pas oublier que:
- beaucoup de jours (avec données manquantes) ont été supprimés du dataset
- certaines valeurs de puissance produite ou prédite négatives ont été ramenées à 0
- les valeurs manquantes localement (pas une journée complète) ont été remplacées par la moyenne de la variable correspondante sur tout le dataset

In [ ]:
fig = px.line(metrics_df, x='start_ts', y="n_mae")
fig.update_yaxes(title_text='nMAE (%)')
fig.show()

La nMAE maximale sur la période est égale à 7.33%, ce qui reste faible et qui confirme une prédiction relativement correcte de la production des différents actifs.

Elle descend même à des valeurs proches de 0% les 16 et 31 octobre.

In [ ]:
fig = px.line(metrics_df, x='start_ts', y="cout_ecarts")
fig.update_yaxes(title_text='coût des écarts (€)')
fig.show()

Le coût des écarts prend quelque fois des valeurs négatives, ce qui veut dire que pour les jours concernés il est plus intéressant d'avoir mal prédit la production et:
- soit d'avoir revendu le surplus de production au gestionnaire de réseau à un prix plus élevé que le prix day-ahead
- soit d'avoir racheté le manque de production au gestionnaire de réseau à un prix plus bas que le prix day-ahead

Il peut paraître surprenant que le prix de rachat au gestionnaire soit inférieur au prix day-ahead OU que le prix de vente au gestionnaire soit supérieur au prix day-ahead...

Pourtant ceci est bien confirmé par le graphique suivant, sur la journée du 02/10/2021, sur lequel les couleurs représentent les variables suivantes:
- en bleu le prix day_ahead
- en rouge le prix de vente au gestionnaire
- en vert le pris de rachat au gestionnaire

La courbe bleue n'est pas toujours située entre la rouge et la verte.

Ceci implique qu'il est intéressant, à certains moment, de:
- simplement acheter de l'électricité au gestionnaire pour la revendre au prix day-ahead (sans avoir besoin de produire!)
- vendre l'électricité au gestionnaire plutôt qu'au prix day-ahead

In [ ]:
day_one = final_data.head(48)
fig = px.line(day_one, x="start_ts",y=["prix_day_ahead","pos_imb_price", "neg_imb_price"])
fig.update_yaxes(title_text='coût des écarts (€)')
fig.show()